In [1]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
import tensorflow.keras as keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, UpSampling2D, Flatten, BatchNormalization, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras import optimizers
from tensorflow.keras.datasets import cifar100
import tensorflow as tf
from keras.utils import np_utils
import numpy as np
import matplotlib.pyplot as plt
import time
from skimage.transform import resize
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator

In [2]:
num_classes = 100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()

169017344/169001437 [==============================] - 6s 0us/step


In [4]:
x_val = x_train[40000:50000, ]
x_train = x_train[:40000, ]
y_val = y_train[40000:50000]
y_train = y_train[:40000]
print(x_train.shape, y_train.shape)

(40000, 32, 32, 3) (40000, 1)


In [5]:
import cv2
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for keras'
    def __init__(self, images , labels = None,  batch_size = 64,
                 dim = (224, 224), n_classes = 100,
                 shuffle = True):
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.dim = dim
        self.n_classes = n_classes
        self.shuffle = shuffle
        
        self.indexes = np.arange(self.images.shape[0]) # (0,1,2,3,...,40000)

        self.on_epoch_end()
        
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        
        if self.shuffle == True:
            np.random.shuffle(self.indexes) 
            
    def __len__(self):
        'Denotes the number of batches per epoch'
        return self.images.shape[0] // self.batch_size 
        
    def __getitem__(self, index):
        'Generate batch of data in position index'
        # index * batch_size : (index +1) *batch_size

        batch_indexes = self.indexes[ index*self.batch_size:(index+1)*self.batch_size]
        
        images =[]
        labels =[] 

        for i in batch_indexes:

          img_i = self.images[i] 
          img_i = cv2.resize(img_i , self.dim )
          img_i = (img_i) / 127. - 1 

          # augm.... 
          images.append(img_i)

          label_i = self.labels[i]
          labels.append(label_i)
      
        images = np.stack(images) # batch_size, 224,224,3
        labels = np_utils.to_categorical(labels, self.n_classes)

        return images, labels 

In [6]:
resnet_model = ResNet50(weights='imagenet', include_top=False,
                        input_shape=(224,224,3))
avg = GlobalAveragePooling2D()(resnet_model.output)

den1 = Dense(256, activation=None)(avg)
bn = BatchNormalization()(den1)
relu = tf.keras.layers.Activation('relu')(bn)

drop1 = Dropout(0.3)(relu)
den2 = Dense(num_classes, activation='softmax')(drop1)

model = tf.keras.Model(inputs = resnet_model.inputs, outputs =[den2])


94781440/94765736 [==============================] - 2s 0us/step


In [ ]:
for layer in resnet_model.layers:
  if isinstance(layer, BatchNormalization):
    layer.trainable = True
  else:
    layer.trainable = False
model.summary()

In [8]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
data_gen = DataGenerator(
    x_train,
    y_train,
    batch_size = 64,
    dim = (224, 224), n_classes = 100,
    shuffle = True
)
val_gen  = DataGenerator(
    x_val,
    y_val,
    batch_size = 64,
    dim = (224, 224), n_classes = 100,
    shuffle = False
)

Freeze pre-train and warm up top layers

In [ ]:
t=time.time()
bt_size = 64 
Epoch = 6
historytemp = model.fit(data_gen,
                        validation_data=val_gen,
                        epochs=Epoch)
print('Training time: %s' % (time.time() - t))

Epoch 1/6
625/625 [==============================] - 695s 1s/step - loss: 1.9012 - accuracy: 0.5171 - val_loss: 4.6319 - val_accuracy: 0.1284
Epoch 2/6
625/625 [==============================] - 658s 1s/step - loss: 1.0024 - accuracy: 0.7087 - val_loss: 0.9896 - val_accuracy: 0.7117
Epoch 3/6
625/625 [==============================] - ETA: 0s - loss: 0.7831 - accuracy: 0.7661

In [ ]:
test_gen = DataGenerator(
    x_test,
    y_test,
    batch_size = 64,
    dim = (224, 224), n_classes = 100,
    shuffle = False
)


Unfreeze pre-train model, update entire model

In [ ]:
# unfreeze weights
for layer in resnet_model.layers:
    layer.trainable = True

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=2e-4),
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor = 'val_loss', mode = 'min',
                   patience = 8, restore_best_weights = True, verbose = 1)

In [ ]:
hist = model.fit(data_gen,
                        validation_data=val_gen,
                  epochs=25,
                  callbacks=[es],
                  verbose=1
                  )


In [ ]:
model.save_weights("./model.h5")

In [ ]:
model.load_weights("./model.h5")

In [ ]:
model.evaluate(test_gen)

156/156 [==============================] - 68s 430ms/step - loss: 5.5458 - accuracy: 0.0100


[5.54579496383667, 0.010016025975346565]

In [ ]:
image_test, label_test = test_gen[0]

In [ ]:
output = model.predict(image_test)

Process Keras_worker_ForkPoolWorker-13:
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
Process Keras_worker_ForkPoolWorker-14:
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/

array([[0.01997674, 0.00173548, 0.00163264, ..., 0.00033187, 0.00428614,
        0.04355282],
       [0.020335  , 0.00178458, 0.00168366, ..., 0.00034463, 0.00427997,
        0.0444195 ],
       [0.01955553, 0.00176332, 0.00160603, ..., 0.0003308 , 0.00411417,
        0.04283148],
       ...,
       [0.02192933, 0.00180662, 0.00169109, ..., 0.00033436, 0.00440127,
        0.04498729],
       [0.01863182, 0.00183937, 0.001633  , ..., 0.0003647 , 0.00406368,
        0.04145779],
       [0.02076144, 0.00179019, 0.0016678 , ..., 0.00033747, 0.00433673,
        0.04419092]], dtype=float32)